# Import the necessary libraries

In [2]:
from glove import Corpus, Glove
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import datetime,time
from flashtext.keyword import KeywordProcessor
from nltk.stem import WordNetLemmatizer
from string import punctuation
import re
import pandas as pd
import string
import multiprocessing
from nltk.corpus import stopwords
from flashtext.keyword import KeywordProcessor
from sklearn.model_selection import train_test_split
from sklearn import metrics

# libraries for dataset preparation, feature engineering, model training 
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
#from utils.language_models import WordVectorEmbedding
from gensim.test.utils import datapath, get_tmpfile
#import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


# Import spooky author data from csv

In [3]:
df=pd.read_csv('train.csv',encoding='utf-8')
df.head()
sen=df['text']

# Preprocess data

In [9]:
# Define a function for removing regex
def regex_filtering(text):
        if text:
            #removing all email metadata fix it for email terms only
            text=re.sub(r"^(sender|to|copy|from|sent|subject|date|cc|e|von|datum|an|importance|bcc):.*$"," ",text,flags=re.M)
            #removing all mail ids
            text=re.sub(r"\S*@\S*\s?"," ",text)
            #removing all links
            text=re.sub(r"(((https?|ftp|file):\/\/)|www\\.)\\S+", ' ', text, flags=re.MULTILINE)
            text=re.sub(r"\w*\.\w{1,4}", '', text, flags=re.MULTILINE)
            #removing all non word character
            text=re.sub(r"([^a-zA-Z0-9\\u00C0-\\u00FF@]|[Ã£Ã¢])+",' ',text)
            #removing words with numbers 
            text=re.sub(r'\w*\d\w*', ' ', text)
            #removing single characters
            text=re.sub(r'\b\S{1}\s+',' ',text)
            #removing words with repeating characters
            text=re.sub(r'\b(\w)\1{1,}\s+',' ',text)
            #removing punkt
            text = text.translate(str.maketrans('','',string.punctuation))
            #removing extra whitespace
            text=re.sub(r"\s\s+",' ',text)
            #removing repeating words
            text=re.sub(r"(\w+\s+)\1{1,}",' ',text)
            #removing whitespaces
            text=text.strip()
            return text

In [10]:
#Tokenize Terms and remove stopwords
def tokenize_term(x):
        predefined_stopwords='horror perfectly'
        english_stopwords=stopwords.words("english")
        stopwords_list=(list(predefined_stopwords.split(' '))+english_stopwords)         
        keyword_processor_stopwords = KeywordProcessor()
        for each in stopwords_list:
            keyword_processor_stopwords.add_keyword(each,' ')   
        sentence=keyword_processor_stopwords.replace_keywords(x)
        return sentence.strip()

In [11]:
#Combined function
def preprocess(text):
    return regex_filtering(tokenize_term(text))

In [8]:
#Apply function preprocess and save result as new column
df['text_preprocessed']=df['text'].apply(preprocess)
texts=df['text_preprocessed']

In [13]:
df.head()

,id,text,author,text_preprocessed
0,id26305,"This process, however, afforded me no means of...",EAP,process however afforded means ascertaining di...
1,id17569,It never once occurred to me that the fumbling...,HPL,never occurred fumbling might mere mistake
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,left hand gold snuff box capered hill cutting ...
3,id27763,How lovely is spring As we looked from Windsor...,MWS,lovely spring looked Windsor Terrace sixteen f...
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,Finding nothing else even gold Superintendent ...


In [14]:
#Lemmatize different forms of words
def lemmatize_doc(dataframe):
    wordnet_lemmatizer = WordNetLemmatizer() 
    lemmatized = [[wordnet_lemmatizer.lemmatize(word,pos='v') for word in word_tokenize(s)]
                  for s in dataframe['text_preprocessed']]
    return lemmatized
lemma_data=lemmatize_doc(df)
df['text_lemmatized']=[" ".join(i) for i in lemma_data]
df.head()

,id,text,author,text_preprocessed,text_lemmatized
0,id26305,"This process, however, afforded me no means of...",EAP,process however afforded means ascertaining di...,process however afford mean ascertain dimensio...
1,id17569,It never once occurred to me that the fumbling...,HPL,never occurred fumbling might mere mistake,never occur fumble might mere mistake
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,left hand gold snuff box capered hill cutting ...,leave hand gold snuff box caper hill cut manne...
3,id27763,How lovely is spring As we looked from Windsor...,MWS,lovely spring looked Windsor Terrace sixteen f...,lovely spring look Windsor Terrace sixteen fer...
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,Finding nothing else even gold Superintendent ...,Finding nothing else even gold Superintendent ...


# Train model

## Defining model path

In [15]:
###Saving models with timestamp
import datetime,time

path="model/"
epochs=50
st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d')+"_"
model_file_path =  path + "spooky_author_glove_embedding_model_"+str(st) + str(epochs) + "_epoch.model"
glove_vectors_path = path + "spooky_author_glove_embedding_model_" + str(st)+ str(epochs) + "_epoch.txt"
#glove_to_w2v_path = path + "spooky_author_glove_embedding_model_" +str(st)  + "_glove_to_word2vec.txt"
print (model_file_path)
print (glove_vectors_path)
#print(glove_to_w2v_path)

model/spooky_author_glove_embedding_model_2019-02-17_50_epoch.model
model/spooky_author_glove_embedding_model_2019-02-17_50_epoch.txt


## Set hyper-parameters for model training

In [16]:
#### Parameters for glove model
window = 5         ##### Contect window of every sentence it has to take
no_components=50  ##### No.of Dimiensions or size of vectors
learning_rate=0.05 ##### Learning rate or alpha
no_threads=84      ##### Multiprocessing ( specify the cores)
epochs=50

## Train glove embedding and save it

In [20]:
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(lemma_data, window=window)

#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=no_components, learning_rate=learning_rate)
 
glove.fit(corpus.matrix, epochs=epochs, no_threads=no_threads, verbose=False)
glove.add_dictionary(corpus.dictionary)
glove.save(model_file_path)
print('Glove embedding model trained successfully. \nModel saved in the following path : {}'.format(model_file_path))

Glove embedding model trained successfully. 
Model saved in the following path : model/spooky_author_glove_embedding_model_2019-02-17_50_epoch.model


## Save model in vector format

In [22]:
# Save model in vector format
word_vectors = glove.word_vectors

# Writing a vector file 
#Sample word : word1 0.2178995609092781 -0.3523739278544217 -0.14607002613618192 -0.0019162911041566608 

words = list(glove.dictionary)
word_vectors_for_words  = word_vectors.tolist()
glove_vectors = [m + " " + str(" ".join(str(item) for item in n)) for m,n in zip(words,word_vectors_for_words)] 

In [23]:
with open(glove_vectors_path,"w+") as fp:
    for line in glove_vectors:
        fp.write(str(line))
        fp.write("\n")
fp.close()

In [24]:
df.head()

,id,text,author,text_preprocessed,text_lemmatized
0,id26305,"This process, however, afforded me no means of...",EAP,process however afforded means ascertaining di...,process however afford mean ascertain dimensio...
1,id17569,It never once occurred to me that the fumbling...,HPL,never occurred fumbling might mere mistake,never occur fumble might mere mistake
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,left hand gold snuff box capered hill cutting ...,leave hand gold snuff box caper hill cut manne...
3,id27763,How lovely is spring As we looked from Windsor...,MWS,lovely spring looked Windsor Terrace sixteen f...,lovely spring look Windsor Terrace sixteen fer...
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,Finding nothing else even gold Superintendent ...,Finding nothing else even gold Superintendent ...


## Remove null rows present after preprocessing

In [25]:
filter_=df['text_lemmatized'].apply(lambda x:True if not x else False)

In [26]:
print("Before removing empty strings {}".format(filter_.sum()))
df=df[~filter_]
filter_=df['text_lemmatized'].apply(lambda x:True if not x else False)
print("After removing empty strings {}".format(filter_.sum()))

Before removing empty strings 6
After removing empty strings 0


In [27]:
df=df[~filter_]

## Find average vector of each sentence from the trained glove vector embeddings

In [29]:
def vectorize(list_of_strings,vector_size):
    global model
    array_len=len(list_of_strings)
    vector=np.array([0]*vector_size,dtype='float64')
    for text in list_of_strings:
        try:
            vector+=word_vectors[glove.dictionary[text]]
        except KeyError:
            vector+=np.array([0]*vector_size,dtype='float64')
    #Vectorize avrg
    vector_to_return=vector/array_len
    return vector_to_return

In [30]:
vectorized_sentences=[vectorize(i,50) for i in df['text_lemmatized']]
df['vectorized_sentences']=vectorized_sentences
df.head()

,id,text,author,text_preprocessed,text_lemmatized,vectorized_sentences
0,id26305,"This process, however, afforded me no means of...",EAP,process however afforded means ascertaining di...,process however afford mean ascertain dimensio...,"[-0.001400567117661585, -0.001340283304660479,..."
1,id17569,It never once occurred to me that the fumbling...,HPL,never occurred fumbling might mere mistake,never occur fumble might mere mistake,"[-0.0009347250675251107, -0.000693032438372496..."
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,left hand gold snuff box capered hill cutting ...,leave hand gold snuff box caper hill cut manne...,"[-0.002165417531272407, -0.001456710128963926,..."
3,id27763,How lovely is spring As we looked from Windsor...,MWS,lovely spring looked Windsor Terrace sixteen f...,lovely spring look Windsor Terrace sixteen fer...,"[-0.0013816478842009468, -0.000885050645054020..."
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,Finding nothing else even gold Superintendent ...,Finding nothing else even gold Superintendent ...,"[-0.0015449137596513203, -0.001517233931621229..."


## Cast average sentence vectors to pandas series for easier processing

In [31]:
#Fit vectors
X_vec=df['vectorized_sentences'].apply(pd.Series)

In [32]:
X_vec.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.001401,-0.001340,-0.002509,0.001066,-0.001001,-0.000061,-2.750992e-04,0.004114,0.000239,-0.000145,...,-0.000506,-0.001195,-0.001706,0.001332,0.000913,0.001430,-0.001371,-0.000020,-0.003701,-0.002175
1,-0.000935,-0.000693,-0.001614,0.000486,-0.000376,0.000378,-1.808857e-04,0.002598,-0.000266,0.000114,...,-0.000360,-0.000422,-0.001358,0.000485,0.000522,0.000843,-0.001231,0.000085,-0.002565,-0.001393
2,-0.002165,-0.001457,-0.003180,0.001160,-0.001349,-0.000094,3.811707e-04,0.005479,-0.000086,-0.000541,...,-0.000505,-0.001794,-0.002524,0.001929,0.001143,0.001242,-0.001695,-0.000470,-0.004260,-0.002887
3,-0.001382,-0.000885,-0.002176,0.000920,-0.000709,-0.000259,-9.933850e-08,0.003671,0.000306,-0.000263,...,-0.000190,-0.001191,-0.001608,0.001279,0.000768,0.000949,-0.001063,-0.000129,-0.002944,-0.002089
4,-0.001545,-0.001517,-0.003125,0.001515,-0.001025,-0.000484,-8.625801e-04,0.004846,0.001085,0.000059,...,-0.000441,-0.001590,-0.001628,0.001658,0.001080,0.002108,-0.001362,0.000179,-0.004685,-0.002941


## Use Label-encoder to transform author class labels to machine understandable number inputs

In [33]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['author_encoded']=le.fit_transform(df['author'])
df.head()

,id,text,author,text_preprocessed,text_lemmatized,vectorized_sentences,author_encoded
0,id26305,"This process, however, afforded me no means of...",EAP,process however afforded means ascertaining di...,process however afford mean ascertain dimensio...,"[-0.001400567117661585, -0.001340283304660479,...",0
1,id17569,It never once occurred to me that the fumbling...,HPL,never occurred fumbling might mere mistake,never occur fumble might mere mistake,"[-0.0009347250675251107, -0.000693032438372496...",1
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,left hand gold snuff box capered hill cutting ...,leave hand gold snuff box caper hill cut manne...,"[-0.002165417531272407, -0.001456710128963926,...",0
3,id27763,How lovely is spring As we looked from Windsor...,MWS,lovely spring looked Windsor Terrace sixteen f...,lovely spring look Windsor Terrace sixteen fer...,"[-0.0013816478842009468, -0.000885050645054020...",2
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,Finding nothing else even gold Superintendent ...,Finding nothing else even gold Superintendent ...,"[-0.0015449137596513203, -0.001517233931621229...",1


In [34]:
#Let's check the dataframe information
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19573 entries, 0 to 19578
Data columns (total 7 columns):
id                      19573 non-null object
text                    19573 non-null object
author                  19573 non-null object
text_preprocessed       19573 non-null object
text_lemmatized         19573 non-null object
vectorized_sentences    19573 non-null object
author_encoded          19573 non-null int64
dtypes: int64(1), object(6)
memory usage: 1.2+ MB


## Prepare data to feed into different ML algorithms

In [36]:
#Change to vector
texts=X_vec
labels=df['author_encoded']

## Split data into train-test set

In [37]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.1, random_state=0)

## Define a function to generate accuracy of different models on the fly

In [39]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, y_test)

In [40]:
print('Shape of X_train {} and X_test {}'.format(len(X_train),len(X_test)))
print('Shape of y_train {} and y_test {}'.format(len(y_train),len(y_test)))

Shape of X_train 17615 and X_test 1958
Shape of y_train 17615 and y_test 1958


In [41]:
# Random Forest Classifier
accuracy = train_model(ensemble.RandomForestClassifier(), X_train, y_train, X_test)
print ("Accuracy Random Forest classifier, with glove embeddings: "+ str(accuracy))

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy Random Forest classifier, with glove embeddings: 0.3769152196118488


In [42]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), X_train, y_train, X_test)
print ("Accuracy of SVC classifier, with glove embeddings: "+ str(accuracy))

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Accuracy of SVC classifier, with glove embeddings: 0.41419816138917265


In [43]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), X_train, y_train, X_test)
print ("Accuracy of Logistic Regression classifier, with glove embeddings: "+ str(accuracy))

Accuracy of Logistic Regression classifier, with glove embeddings: 0.41419816138917265


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
